In [1]:
import os
import pandas as pd
import numpy as np

import torch
import torch.nn as nn
from torch.utils.data import DataLoader

import torchvision
import torchvision.transforms as T

# Evaluation - Linear Probing

In [11]:
train_data = torchvision.datasets.STL10('./content',split='train',download=True,transform=T.ToTensor())
test_data = torchvision.datasets.STL10('./content',split='test',download=True,transform=T.ToTensor())

batch_size = 16
train_data_loader = DataLoader(train_data,batch_size)
test_data_loader = DataLoader(test_data,batch_size)

Files already downloaded and verified
Files already downloaded and verified


In [12]:
class ResNet18(nn.Module):
    def __init__(self,):
        super(ResNet18, self).__init__()
        resnet18 = torchvision.models.resnet18()
        resnet18.fc = nn.Linear(resnet18.fc.in_features,128)
        self.base_model = resnet18
        self.classifier = nn.Linear(128,10)

    def forward(self,x):
      features = self.base_model(x)
      predictions = self.classifier(features)
      return predictions

test_model = ResNet18()

"""
Load the pretrained weights (comment the following line to check the linear probing accuray with random weights)
"""
test_model.base_model.load_state_dict(torch.load('/content/paws_weights.pth'))

for p in test_model.base_model.parameters(): p.requires_grad=False
print(sum(p.numel() for p in test_model.parameters() if p.requires_grad))

1290


In [13]:
def train_one_epoch(train_data_loader,model,optimizer,loss_fn):
    
    ### Local Parameters
    epoch_loss = []
    sum_correct_pred = 0
    total_samples = 0
    
    model.train()

    ###Iterating over data loader
    for images, labels in train_data_loader:
        
        #Loading images and labels to device
        images = images.to(device)
        labels = labels.to(device)
        
        #Reseting Gradients
        optimizer.zero_grad()
        
        #Forward
        preds = model(images)
        #Calculating Loss
        _loss = loss_fn(preds, labels)
        loss = _loss.item()
        epoch_loss.append(loss)
        
        #Backward
        _loss.backward()
        optimizer.step()

        sum_correct_pred += (torch.argmax(preds,dim=1) == labels).sum().item()
        total_samples += len(labels)

    acc = round(sum_correct_pred/total_samples,4)*100
    ###Acc and Loss
    epoch_loss = np.mean(epoch_loss)
    
    return epoch_loss, acc

def val_one_epoch(val_data_loader, model,loss_fn):
    
    ### Local Parameters
    epoch_loss = []
    sum_correct_pred = 0
    total_samples = 0

    model.eval()
    
    ###Iterating over data loader
    with torch.no_grad():
      for images, labels in val_data_loader:

        #Loading images and labels to device
        images = images.to(device)
        labels = labels.to(device)
      
        #Forward
        preds = model(images)
        
        #Calculating Loss
        _loss = loss_fn(preds, labels)
        loss = _loss.item()
        epoch_loss.append(loss)

        sum_correct_pred += (torch.argmax(preds,dim=1) == labels).sum().item()
        total_samples += len(labels)

    acc = round(sum_correct_pred/total_samples,4)*100
    ###Acc and Loss
    epoch_loss = np.mean(epoch_loss)

    return epoch_loss, acc

In [14]:
# Optimizer
test_optimizer = torch.optim.Adam(test_model.parameters(), lr = 3e-4)
#Loss Function
test_loss_fn = nn.CrossEntropyLoss()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [15]:
test_model.to(device)

for epoch in range(15):
    
    ###Training
    loss, acc = train_one_epoch(train_data_loader,test_model,test_optimizer,test_loss_fn)
    print("\nEpoch {}".format(epoch+1))
    print("Train Acc : {}".format(round(acc, 4)))
    
    ###Validation
    val_loss, val_acc = val_one_epoch(test_data_loader,test_model,test_loss_fn)
    print("Test Acc : {}".format(round(val_acc, 4)))


Epoch 1
Train Acc : 47.64
Test Acc : 56.15

Epoch 2
Train Acc : 56.5
Test Acc : 56.81

Epoch 3
Train Acc : 57.26
Test Acc : 57.35

Epoch 4
Train Acc : 57.6
Test Acc : 57.36

Epoch 5
Train Acc : 57.92
Test Acc : 57.47

Epoch 6
Train Acc : 58.06
Test Acc : 57.46

Epoch 7
Train Acc : 58.38
Test Acc : 57.54

Epoch 8
Train Acc : 58.42
Test Acc : 57.8

Epoch 9
Train Acc : 58.5
Test Acc : 57.87

Epoch 10
Train Acc : 58.64
Test Acc : 57.94

Epoch 11
Train Acc : 58.7
Test Acc : 57.94

Epoch 12
Train Acc : 58.9
Test Acc : 58.0

Epoch 13
Train Acc : 59.02
Test Acc : 58.09

Epoch 14
Train Acc : 59.12
Test Acc : 58.16

Epoch 15
Train Acc : 59.16
Test Acc : 58.29
